In [36]:
#imports
import pandas as pd
import numpy as np
import random
import math

In [37]:
#programm the game itself with 5 players
class game:

    #generally states looks like {2 of Diamonds at human player: 0 or 1, 2 of Diamonds at AI player: 0 or 1, 2 of Diamonds shown: 0 or 1, 2 of diamonds at dealer: 0 or 1}

    def __init__(self):
        self.outcome = 0
        self.state = self.generate_start_state()
        self.cards = self.generate_card_deck()
        self.future_cards = [] #cards that will be dealed after the player made their desicion, represented as the index of the corresponding card
        self.deal_cards() #deals cards
        #self.print_state()
        self.stay_in = True #self.ask_player_to_call() #asks player to fold or call
        if self.stay_in: #if player calls deal remaining two cards and evaluate outcome
            self.deal_remaining_two()
            self.outcome = self.evaluate_outcome()
        else:
            #if player folds return the outcome is always -20
            self.outcome = -20

    #generate a card decks with descriptions of the cards to reduce redundancy
    def generate_card_deck(self):
        card_list = []
        for symbol in ["Diamonds","Hearts","Pick","Clubs"]:
            for number in ["2","3","4","5","6","7","8","9","10","J","Q","K","A"]:
                card_list.append(number+" of " +symbol)
        return card_list

    #generates the start states where everything is 0
    def generate_start_state(self):
        state = np.zeros((52*4,))
        state = np.round(state).astype(int)
        return state
    
    #prints out the current state of the game that is known to the human player if show_dealer_cards = False, otherwise shows whole state
    def print_state(self, show_dealer_cards = False):
        card_list = self.cards
        dealed_to_human_player = []
        dealed_to_ai_player = []
        openly_shown = []
        dealed_to_dealer = []
        for idx, val in np.ndenumerate(self.state):
            idx = idx[0]
            if val == 1:
                if idx % 4 == 0:
                    dealed_to_human_player.append(card_list[idx//4])
                elif idx % 4 == 1:
                    dealed_to_ai_player.append(card_list[idx//4])
                elif idx % 4 == 2:
                    openly_shown.append(card_list[idx//4])
                elif idx % 4 == 3:
                    dealed_to_dealer.append(card_list[idx//4])
        print("DEALED TO HUMAN PLAYER:")
        for card in dealed_to_human_player:
            print(card)
        print("--------------------------------------")
        print("DEALED TO BOT PLAYER:")
        for card in dealed_to_ai_player:
            print(card)
        print("--------------------------------------")
        print("CARDS IN THE MIDDLE:")
        for card in openly_shown:
            print(card)
        print("--------------------------------------")
        if show_dealer_cards:
            print("DEALED TO DEALER:")
            for card in dealed_to_dealer:
                print(card)
            print("--------------------------------------")
            print("CARDS NOT SHOWN:")
            for card in set(card_list).difference((set(dealed_to_human_player).union(set(dealed_to_ai_player), set(openly_shown), set(dealed_to_dealer)))):
                print(card)
        else:
            print("CARDS NOT SHOWN:")
            for card in set(card_list).difference((set(dealed_to_human_player).union(set(dealed_to_ai_player), set(openly_shown)))):
                print(card)

    #simple function that deals the initial cards and also predifines the cards that will be drawn later
    def deal_cards(self):
        cards_in_deck = [i for i in range(52)]
        #deal to human player
        for i in range(2):
            card = random.choice(cards_in_deck)
            self.state[4*card] = 1
            cards_in_deck.remove(card)
        #deal to bot player
        for j in range(10):
            card = random.choice(cards_in_deck)
            self.state[4*card+1] = 1
            cards_in_deck.remove(card)
        #deal to openly shown on the table
        for t in range(3):
            card = random.choice(cards_in_deck)
            self.state[4*card+2] = 1
            cards_in_deck.remove(card)
        #deal to dealer
        for h in range(2):
            card = random.choice(cards_in_deck)
            self.state[4*card+3] = 1
            cards_in_deck.remove(card)
        #preset the cards that will be dealed out after the player made their desicion
        for k in range(2):
            card = random.choice(cards_in_deck)
            self.future_cards.append(card)
            cards_in_deck.remove(card)

    #function to deal the remaining two cards
    def deal_remaining_two(self):
        for card in self.future_cards:
            self.state[4*card+2] = 1

    #asks the player wheter they want to continue playing based on the dealed cards
    def ask_player_to_call(self):
        cont = input("Do you want to continue? (Y/N)? ")
        if cont=="Y":
            return True
        else:
            return False
    
    #evaluates the outcome of the game 
    def evaluate_outcome(self):
        #get all cards that are either in the middle or at the human player
        player_score = 0 #2=Highest card 2, 3=Highest card 3, ..., 15=Higest pair 2, 16=Highest pair 3,..., etc
        dealer_score = 0
        open_cards_index_player = [] #holds the 7 cards as indexes that have to be checked for the player
        open_cards_words_player = [] #holds the 7 cards as words that have to be checked for the player
        open_cards_index_dealer = [] #holds the 7 cards as indexes that have to be checked for the dealer
        open_cards_words_dealer = [] #holds the 7 cards as words that have to be checked for the dealer
        for idx, val in np.ndenumerate(self.state):
            idx = idx[0]
            if val == 1:
                if idx%4==2 or idx%4==0:
                    open_cards_index_player.append(idx//4)
                if idx%4==2 or idx%4==3:
                    open_cards_index_dealer.append(idx//4)
        for card_index in open_cards_index_player:
            open_cards_words_player.append(self.cards[card_index])
        for card_index in open_cards_index_dealer:
            open_cards_words_dealer.append(self.cards[card_index])
        #check for highest card max value here is 14
        player_score = self.max(player_score,self.check_for_highest_card(open_cards_words_player))
        dealer_score = self.max(dealer_score,self.check_for_highest_card(open_cards_words_dealer))
        #check for highest pair max value here is 27
        player_score = self.max(player_score,self.check_for_highest_pair(open_cards_words_player))
        dealer_score = self.max(dealer_score,self.check_for_highest_pair(open_cards_words_dealer))

        #check for highest two pairs (max value 195)/Three of a kind (max value 208)
        if player_score > 14: #check for wheter the player actually has a pair
            player_score = self.max(player_score,self.check_for_two_highest_pairs(open_cards_words_player))
            player_score = self.max(player_score,self.check_for_three_of_a_kind(open_cards_words_player)) 
        if dealer_score > 14: #check for wheter the player actually has a pair
            dealer_score = self.max(dealer_score,self.check_for_two_highest_pairs(open_cards_words_dealer))
            dealer_score = self.max(player_score,self.check_for_three_of_a_kind(open_cards_words_dealer)) 
        
        #check for highest four of a kind (max value 399)/Full house (max value 386)
        if player_score > 195: #check for wheter the player actually has three of a kind
            player_score = self.max(player_score,self.check_for_full_house(open_cards_words_player))
            player_score = self.max(player_score,self.check_for_four_of_a_kind(open_cards_words_player))
        if dealer_score > 195:
            dealer_score = self.max(dealer_score,self.check_for_full_house(open_cards_words_dealer))
            dealer_score = self.max(dealer_score,self.check_for_four_of_a_kind(open_cards_words_dealer))

        #check for straight max value is 217
        player_score = self.max(player_score,self.check_for_straight(open_cards_words_player))
        dealer_score = self.max(dealer_score,self.check_for_straight(open_cards_words_dealer))
        #check for flush max value is 218
        player_score = self.max(player_score,self.check_for_flush(open_cards_words_player))
        dealer_score = self.max(dealer_score,self.check_for_flush(open_cards_words_dealer))
        #check for straight flush (max value 407)/royale flush (max value 408)
        player_score = self.max(player_score,self.check_for_flush(open_cards_words_player))
        dealer_score = self.max(dealer_score,self.check_for_flush(open_cards_words_dealer))

        #case 1: Dealer not qualified
        if dealer_score < 17:
            return 20
        #case 2: Dealer wins
        elif dealer_score > player_score:
            return -60
        #case 3: Player wins
        elif dealer_score < player_score:
            #TO-DO check for how much actually is returned for higher cards like e.g. full hosue
            return 60
        #case player and dealer tie
        elif dealer_score == player_score:
            return 0
        
        
        

    #functions that calculates the maximum of two numbers
    def max(self, old, new):
        if old<new:
            return new
        else:
            return old
        
    #helper functions to help to check for two, three and four of a kind
    #checks number of strings in strings which include a certain substring
    def count_strings_with_substring(self, strings, substring):
        count = 0
        for s in strings:
            if substring in s:
                count += 1
        return count
    
    #check for two of a kind
    def exactly_two_strings_with_substring(self, strings, substring):
        count = self.count_strings_with_substring(strings, substring)
        return count == 2
    
    #check for three of a kind
    def exactly_three_strings_with_substring(self, strings, substring):
        count = self.count_strings_with_substring(strings, substring)
        return count == 3
    
    #check for four of a kind
    def exactly_four_strings_with_substring(self, strings, substring):
        count = self.count_strings_with_substring(strings, substring)
        return count == 4

    #check for five of a kind (used for flush)
    def exactly_five_strings_with_substring(self, strings, substring):
        count = self.count_strings_with_substring(strings, substring)
        return count == 5
    
    #helper function that selects only strings in list that have a certain substring
    def filter_strings_according_to_substrings(self, strings, substring):
        return_list = []
        for s in strings:
            if substring in s:
                return_list.append(s)
        return return_list

    #assign value according to highest card
    def check_for_highest_card(self, open_cards_words):
        val = 0
        for card in open_cards_words:
            if "A" in card:
                val = self.max(14,val)
            elif "K" in card:
                val= self.max(13,val)
            elif "Q" in card:
                val = self.max(12,val)
            elif "J" in card:
                val = self.max(11,val)
            elif "10" in card:
                val = self.max(10,val)
            elif "9" in card:
                val = self.max(9,val)
            elif "8" in card:
                val = self.max(8,val)
            elif "7" in card:
                val = self.max(7,val)
            elif "6" in card:
                val = self.max(6,val)
            elif "5" in card:
                val = self.max(5,val)
            elif "4" in card:
                val = self.max(4,val)
            elif "3" in card:
                val = self.max(3,val)
            elif "2" in card:
                val = self.max(2,val)
        return val
    
    def check_for_highest_pair(self, open_cards_words):
        val = 0
        if self.exactly_two_strings_with_substring(open_cards_words,"A"):
            val = self.max(27,val)
        elif self.exactly_two_strings_with_substring(open_cards_words,"K"):
            val = self.max(26,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"Q"):
            val = self.max(25,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"J"):
            val = self.max(24,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"10"):
            val = self.max(23,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"9"):
            val = self.max(22,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"8"):
            val = self.max(21,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"7"):
            val = self.max(20,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"6"):
            val = self.max(19,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"5"):
            val = self.max(18,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"4"):
            val = self.max(17,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"3"):
            val = self.max(16,val) 
        elif self.exactly_two_strings_with_substring(open_cards_words,"2"):
            val = self.max(15,val) 
        return val
    
    #checks for the two highest cards
    def check_for_two_highest_pairs(self,open_cards_words):
        val = 0
        list_of_letters = ["2","3","4","5","6","7","8","9","10","J","Q","K","A"]
        for letter_1_index in range(len(list_of_letters)):
            if self.exactly_two_strings_with_substring(open_cards_words,list_of_letters[letter_1_index]):
                for letter_2_index in range(len(list_of_letters)):
                    if self.exactly_two_strings_with_substring(open_cards_words,list_of_letters[letter_2_index]):
                        val = self.max(val,27+letter_1_index*13+letter_2_index)
        return val

    #check for three of a kind
    def check_for_three_of_a_kind(self,open_cards_words):
        val = 0
        if self.exactly_three_strings_with_substring(open_cards_words,"A"):
            val = self.max(208,val)
        elif self.exactly_three_strings_with_substring(open_cards_words,"K"):
            val = self.max(207,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"Q"):
            val = self.max(206,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"J"):
            val = self.max(205,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"10"):
            val = self.max(204,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"9"):
            val = self.max(203,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"8"):
            val = self.max(202,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"7"):
            val = self.max(201,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"6"):
            val = self.max(200,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"5"):
            val = self.max(199,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"4"):
            val = self.max(198,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"3"):
            val = self.max(197,val) 
        elif self.exactly_three_strings_with_substring(open_cards_words,"2"):
            val = self.max(196,val) 
        return val
    
    #checks wheter there is a straight in the cards
    def check_for_straight(self, open_cards_words):
        val = 0
        open_cards_word_one_word = ""
        for card in open_cards_words:
            open_cards_word_one_word += card
        list_of_letters = ["2","3","4","5","6","7","8","9","10","J","Q","K","A"]
        for ind in range(len(list_of_letters)-4):
            if (list_of_letters[ind] in open_cards_word_one_word 
                and list_of_letters[ind+1] in open_cards_word_one_word 
                and list_of_letters[ind+2] in open_cards_word_one_word
                and list_of_letters[ind+3] in open_cards_word_one_word
                and list_of_letters[ind+4] in open_cards_word_one_word):
                val = 209+ind
        return val

    #checks for flush in the cards
    def check_for_flush(self,open_cards_words):
        val = 0
        symbols = ["Diamonds","Hearts","Pick","Clubs"]
        for symbol in symbols:
            if self.exactly_five_strings_with_substring(open_cards_words,symbol):
                val = 218
        return val
    
    #checks for full house in the cards
    def check_for_full_house(self,open_cards_words):
        val = 0
        list_of_letters = ["2","3","4","5","6","7","8","9","10","J","Q","K","A"]
        for letter_1_index in range(len(list_of_letters)):
            if self.exactly_three_strings_with_substring(open_cards_words,list_of_letters[letter_1_index]):
                for letter_2_index in range(len(list_of_letters)):
                    if self.exactly_two_strings_with_substring(open_cards_words,list_of_letters[letter_2_index]):
                        val = self.max(val,219+letter_1_index*13+letter_2_index)
        return val
    
    #check for four of a kind in the cards
    def check_for_four_of_a_kind(self,open_cards_words):
        val = 0
        if self.exactly_four_strings_with_substring(open_cards_words,"A"):
            val = self.max(399,val)
        elif self.exactly_four_strings_with_substring(open_cards_words,"K"):
            val = self.max(398,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"Q"):
            val = self.max(397,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"J"):
            val = self.max(396,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"10"):
            val = self.max(395,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"9"):
            val = self.max(394,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"8"):
            val = self.max(393,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"7"):
            val = self.max(392,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"6"):
            val = self.max(391,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"5"):
            val = self.max(390,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"4"):
            val = self.max(389,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"3"):
            val = self.max(388,val) 
        elif self.exactly_four_strings_with_substring(open_cards_words,"2"):
            val = self.max(387,val) 
        return val
    
    #check for staright and royale flush
    def check_for_staright_flush(self,open_cards_words):
        val = 0
        #check for flush first
        symbols = ["Diamonds","Hearts","Pick","Clubs"]
        for symbol in symbols:
            if self.exactly_five_strings_with_substring(open_cards_words,symbol):
                flush_symbol = symbol
        open_cards_words = self.filter_strings_according_to_substrings(open_cards_words,flush_symbol) #filter so that only cards with that symbol are in list
        #now check for straight
        open_cards_word_one_word = ""
        for card in open_cards_words:
            open_cards_word_one_word += card
        list_of_letters = ["2","3","4","5","6","7","8","9","10","J","Q","K","A"]
        for ind in range(len(list_of_letters)-4):
            if (list_of_letters[ind] in open_cards_word_one_word 
                and list_of_letters[ind+1] in open_cards_word_one_word 
                and list_of_letters[ind+2] in open_cards_word_one_word
                and list_of_letters[ind+3] in open_cards_word_one_word
                and list_of_letters[ind+4] in open_cards_word_one_word):
                val = 400+ind
        return val
    
    #getter for the outcome of the game
    def get_outcome(self):
        return self.outcome


In [41]:
#test for straight up staying always in 
value = 0
outcome_list = []
for j in range(100000):
    game_1 = game()
    game_2 = game()
    outcome = game_1.get_outcome()
    outcome_list.append(outcome)
print(np.mean(outcome_list))

5.2712
